In [2]:
import sys, os
import numpy as np
import json
import pickle
from typing import Any, Union, Dict, List
import humpack
from humpack import tdict, tlist, tset
from humpack import secure as scr
from humpack import pack, unpack, pack_data, unpack_data, json_pack, json_unpack
from humpack.wrappers import Array

In [3]:
import humpack._lib_info as info

In [6]:
options = tlist(['orange', 'apple'])

['apple', 'orange']

In [7]:
options[:-1]

t['orange']

In [5]:
from humpack import pack, unpack

x = {'one': 1, 1:2, None: ['hello', 123j, {1,3,4,5}]}

p = pack(x) # several standard python types are already packable
assert isinstance(p, dict)
deepcopy_x = unpack(p)
assert repr(x) == repr(deepcopy_x)

from humpack import json_pack, json_unpack # Convert to/from json string

j = json_pack(x)
assert isinstance(j, str)
deepcopy_x = json_unpack(j)
assert repr(x) == repr(deepcopy_x)


from humpack import save_pack, load_pack # Save/load packed object to disk as json file
import os, tempfile

fd, path = tempfile.mkstemp()
try:
    with open(path, 'w') as tmp:
        save_pack(x, tmp)
    with open(path, 'r') as tmp:
        deepcopy_x = load_pack(tmp)
finally:
    os.remove(path)
assert repr(x) == repr(deepcopy_x)


In [14]:
type(j)

str

In [1]:
from humpack import Transactionable

class Account(Transactionable):
    def __init__(self, user, balance=0.):
        super().__init__()
        self._in_transaction = False
        self._shadow_user = None

        self.user = user
        self.balance = balance

    def change(self, delta):

        if self.balance + delta < 0.:
            raise ValueError
        self.balance += delta

    def begin(self):
        # FIRST: begin the transaction in self
        self._shadow_user = self.user.copy(), self.balance # Assuming `user` can be shallow copied.
        self._in_transaction = True

        # THEN: begin transactions in any members that are Transactionable
        if isinstance(self.user, Transactionable):
            self.user.begin()

        # To be extra safe, you could also check `self.balance`, but we'll assume it's always a primitive (eg. float)

    def in_transaction(self):
        return self._in_transaction

    def commit(self):
        # FIRST: commit the transaction in self
        self._in_transaction = False
        self._shadow_user = None

        # THEN: commit transactions in any members that are Transactionable
        if isinstance(self.user, Transactionable):
            self.user.commit()

    def abort(self):
        # FIRST: abort the transaction in self
        if self.in_transaction(): # Note that this call only has an effect if self was in a transaction.
            self.user, self.balance = self._shadow_user

        self._in_transaction = False
        self._shadow_user = None

        # THEN: abort transactions in any members that are Transactionable
        if isinstance(self.user, Transactionable):
            self.user.abort()


In [14]:
a = Account(['Mark', 'Money'] , 0)
a.user, a.balance

(['Mark', 'Money'], 0)

In [22]:
a.begin()
a.in_transaction()

True

In [23]:
a.change(100)
a.user[0] = 'Bill'
a.user, a.balance

(['Bill', 'Money'], 200)

In [24]:
a.abort()

In [25]:
a.user, a.balance

(['Bill', 'Money'], 100)

In [2]:
x = Array(np.random.randn(3))
x

array([-1.41997856,  2.75927546,  1.04447258])

In [3]:
p = json_pack(x)
p

'{"table": {"<>140533281625104": {"_type": "humpack.wrappers.Array", "_data": {"dtype": "float64", "data": "<>140533281146592"}}, "<>140533281146592": {"_data": [-1.4199785551216209, 2.7592754577655887, 1.0444725754363051], "_type": "list"}}, "meta": {}, "head": "<>140533281625104"}'

In [4]:
c = json_unpack(p)
c

array([-1.41997856,  2.75927546,  1.04447258])

In [8]:
x = np.arange(3)
x

array([0, 1, 2])

In [9]:
x = x.astype(object)
x.dtype

dtype('O')

In [ ]:
x.dtype

In [ ]:
class A(object):
    def __init_subclass__(cls, **kwargs):
        print('init', cls, kwargs)
        super().__init_subclass__()
    
class T(object):
    pass

In [ ]:
class B(A, check_type=T):
    pass

In [ ]:
a = np.ndarray.__new__()
a

In [ ]:
t = (3,[])
t

In [ ]:
t[1].append(t)
t

In [ ]:
x = tdict()
x.a = 1
x.x = x
x.l = [tlist(), tset()]
x[100] = '100'
x[None] = 1.2
x.m = None
np.random.seed(1)
x.b = np.random.randn(3).tobytes()
x[b'\xaa'] = 'running'
t = x
t

In [ ]:
x = scr.secure_pack(t, hsh='test')
# x = pack(t)
# x = pickle.dumps(t)
len(x)

In [ ]:
x

In [ ]:
# l = unpack(x)
l = scr.secure_unpack(x, hsh='test')
# l = pickle.loads(x)
l

In [ ]:
x = tdict()
x.a = 1
x.x = x
x.l = [tlist(), tset()]
x[100] = '100'
x[None] = 1.2
x.m = None
x.b = np.random.randn(3).tobytes()
x[b'\xaa'] = 'running'
x

In [ ]:
msg = json.dumps(pack(x)).encode('latin1')
msg

In [ ]:
password = b'password'

kdf = PBKDF2HMAC(
    algorithm=hashes.SHA256(),
    length=32,
    salt=salt,
    iterations=100000,
    backend=default_backend(),
)
key = base64.urlsafe_b64encode(kdf.derive(password))
f = Fernet(key)
token = f.encrypt(msg)
print(len(msg), len(token))
lmsg = f.decrypt(token)
lobj = unpack(json.loads(lmsg.decode('latin1')))
lobj

In [ ]:
salt = b'6FwLrxJb5mTPVwthumpackMASTERsalt'
len(salt)

In [ ]:
f = Fernet(base64.urlsafe_b64encode(salt))

In [ ]:
from crypt import crypt

In [ ]:
crypt('test', salt.decode())

In [ ]:
id(t)

In [ ]:
x = tdict()
x.a = 1
x.x = x
x.l = [tlist(), tset()]
x[100] = '100'
x[None] = 1.2
x.m = None
x.b = np.random.randn(3).tobytes()
x[b'\xaa'] = 'running'
x

In [ ]:
d = pack(x)
d

In [ ]:
j = json.dumps(d)
j

In [ ]:
uj = json.loads(j)
o = unpack(d)
o

In [ ]:
(np.frombuffer(x.b) - np.frombuffer(o.b)).sum() == 0

In [ ]:
from crypt import crypt, mksalt
import getpass
from hmac import compare_digest

In [ ]:
_master_salt = '$6$wGLrx8b5FmTPbhumpack-master-salt'

In [ ]:
ps = 'test'
h = crypt(ps, _master_salt)

In [ ]:
h = crypt(ps, h)
h

In [ ]:
compare_digest(h, crypt(ps, h))

In [ ]:
z = 1.0003+3j + np.pi
x = (z/1e18)*10003e13
x

In [ ]:
complex(str(x)) == x

In [ ]:
humpack.__version__

In [ ]:
P = (type(None), str, float, int, bool)
P

In [ ]:
Union[P]

In [ ]:
humpack.PRIMITIVE

In [ ]:
Union[P]